<a href="https://colab.research.google.com/github/bnriiitb/whisper-fine-tuning/blob/main/model_evaluation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Dec 23 09:53:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:34:00.0 Off |                    0 |
| N/A   43C    P0   304W / 350W |  26514MiB / 32768MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:36:00.0 Off |                    0 |
| N/A   

In [1]:
import numpy as np
import pandas as pd
import os
from IPython import display
import datasets
from datasets.info import DatasetInfosDict
from datasets import load_dataset,concatenate_datasets, DatasetDict, Dataset, Audio

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fleurs_dataset = load_dataset("google/fleurs", "te_in", split="test",num_proc=32)
fleurs_dataset = fleurs_dataset.rename_column("transcription", "text")
fleurs_dataset = fleurs_dataset.cast_column("audio",Audio(sampling_rate=16000))
fleurs_dataset = fleurs_dataset.remove_columns(['id', 'num_samples', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_dataset

Found cached dataset fleurs (/raid/cs20mds14030/.cache/huggingface/datasets/google___fleurs/te_in/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


Dataset({
    features: ['path', 'audio', 'text'],
    num_rows: 472
})

In [3]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import soundfile as sf
import torch
import evaluate
metric = evaluate.load("wer")

# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# def map_to_pred(batch):
    
#     # print(batch['audio'][0]['array'])
    
#     input_features = processor(batch["audio"]["array"], return_tensors="pt",sampling_rate=16000).input_features

#     with torch.no_grad():
#         logits = model(input_features.to("cuda")).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     transcription = processor.batch_decode(predicted_ids, normalize = True)
#     batch['text'] = processor.tokenizer._normalize(batch['text'])
#     batch["transcription"] = transcription
#     return batch

# result = fleurs_dataset.map(map_to_pred)

# wer = load("wer")
# print(wer.compute(predictions=ds["text"], references=ds["transcription"]))

In [5]:
import time
from tqdm import tqdm
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te-146h",language='Telugu')

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text",
#     title="Whisper Small Telugu",
#     description="Realtime demo for Telugu speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████| 967M/967M [01:12<00:00, 13.3MB/s]


In [6]:
device = "cuda:15" if torch.cuda.is_available() else "cpu"

In [7]:
fleurs_dataset[0]

{'path': '/raid/cs20mds14030/.cache/huggingface/datasets/downloads/extracted/2ac244d14fb34cba3d8dc25621891cff63e65f2cb5189e239e867350cda9f3b4/te_in/audio/test/7038975664927295261.wav',
 'audio': {'path': '7038975664927295261.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00052094, 0.00167996,
         0.00159758]),
  'sampling_rate': 16000},
 'text': 'రెండవ సెట్\u200cలో del potroకు ఆధిక్యం లభించినా కూడా ఈ సెట్\u200cలో కూడా 6-6కి చేరుకున్న తర్వాత టై బ్రేక్ అనివార్యం అయ్యింది'}

In [8]:
pipe(fleurs_dataset[0]['audio']['array'])

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': 'రెండవ సెట్లో డెల్పోట్రాకు ఆధిక్యం లభించినా కూడా ఈ సెట్లో కూడా ఆరు ఆనునున్నారుకి చేరుకున్న తర్వాత డైబ్రే గనివార్యం అయ్యింది'}

In [9]:
finetuned_texts = []

for item in tqdm(fleurs_dataset['audio']):
    finetuned_texts.append(pipe(item['array'],language='Telugu')['text'])

finetuned_texts

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 472/472 [2:27:07<00:00, 18.70s/it]


['రెండవ సెట్లో డెల్పోట్రాకు ఆధిక్యం లభించినా కూడా ఈ సెట్లో కూడా ఆరు ఆనునున్నారుకి చేరుకున్న తర్వాత డైబ్రే గనివార్యం అయ్యింది',
 'హాజరైన వారి సంఖ్య ఎంత ఎక్కువంటే సైన్ పీఠర్ స్క్వేడ్లో జరిగిన అంతక్రియలను చూడటానికి అందరికీ సాధ్యపడలేదు',
 'కేవలం కొన్ని స్వల్ప భేదాలు తప్ప కాల్పనిక బృందాలు కూడా సంప్రదాయ బృందాలు వంటి నైపుణ్యం కలిగి ఉంటాయి',
 'ఎయిరోస్మిత్ వారి పర్యటనలో మిగిలిన సంగీత కచేరీలను రద్దు చేసింది',
 'ఈ ప్రమాదం ఎత్తుగా ఉన్న పర్వత భూభాగంలో జరిగిందని ఇంకా శత్రులు రాజేసిన అగ్నిఫలితంగా జరిగిందని నమ్ముతారు',
 'ఇతర నామినేషన్లలో ఉత్తమ చిత్రం దర్శకుడు సినిమాటగ్రఫిక్ ఆస్ట్యూమ్ డిజైన్ ఫిల్మ్ ఎడిటింగ్ అర్జినల్ స్కోర్ ప్రొడక్షన్ డిజైన్ సౌండ్ ఎడిటింగ్ సౌండ్ మిక్సింగ్ మరియు అర్జినల్ క్రీన్ ప్లేజ్ చేర్చబడి ఉన్నాయి',
 'పార్సియన్లు ఉద్రేకంగా మొరటుగా మరియు అహంకారంతో కూడుకొని ఉంటారని పేరు ఉంది',
 'ఈ నగరం దేశంలో నితర నగరాలకు పూర్తిగా విద్ధమైనది ఎందుకంటే అది ఎక్కువగా ఆఫ్రికన్ శైలు కంటే అరబిక్ శైలు కలిగి ఉంటుంది',
 'అటికన్ సిటీ జనాభాసం అరిందందుల మంది నింది ప్రభావించనేది చిన్న స్వతంత దేశం కలిగినా అతి తక్కువ 

In [10]:
finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=fleurs_dataset['text'])
finetuned_wer

35.472325194938

In [ ]:
# !pip install git+https://github.com/openai/whisper.git -q

: 

In [ ]:
# import whisper
# whisper_small_model = whisper.load_model("small",device='cuda')
# whisper_small_model = whisper.load_model("small")

: 

In [ ]:
# !conda install pytorch torchvision torchaudio -c pytorch --yes

: 

In [ ]:
fleurs_dataset[1]['text']

: 

In [ ]:
display.Audio(fleurs_dataset[1]['audio']['array'],rate=16000)

: 

In [ ]:
pipe(fleurs_dataset[1]['audio']['array'])['text']

: 

In [ ]:
# whisper_small_model.transcribe(fleurs_dataset[0]['audio']['array'],language="Telugu")
whisper_small_model.transcribe(fleurs_dataset[1]['audio']['array'].astype(np.float32),language="Telugu")['text']

: 

: 

In [ ]:
whisper_langs = []
whisper_texts = []

for item in tqdm(fleurs_dataset['path']):
    pred = whisper_small_model.transcribe(item,language='Telugu')
    whisper_texts.append(pred['text'])
    whisper_langs.append(pred['language'])
whisper_texts

: 

In [ ]:
import evaluate
metric = evaluate.load("wer")

finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=fleurs_dataset['text'])
whisper_wer = 100 * metric.compute(predictions=whisper_texts, references=fleurs_dataset['text'])

whisper_wer,finetuned_wer

: 

In [4]:
import requests
import json

import base64
import ast

def wav_to_base64string(file_path):
    return base64.b64encode(open(file_path, "rb").read()).decode('utf-8')

url = "https://ai4b-dev-asr.ulcacontrib.org/asr/v1/recognize/te"

def get_payload(base64_audio_content):
    payload = json.dumps({
      "config": {
        "language": {
          "sourceLanguage": "te"
        },
        "transcriptionFormat": {
          "value": "transcript"
        },
        "audioFormat": "wav"
      },
      "audio": [
        {
          "audioContent": f"{base64_audio_content}"
        }
      ]
    })
    return payload

headers = {
  'Content-Type': 'application/json'
}

In [ ]:
BASE_PATH = "/raid/cs20mds14030/telugu_asr/data"
def create_dataset_from_metadata(dataset_name):
    ds = load_dataset('csv', data_files=f"{BASE_PATH}/{dataset_name}/metadata.csv")
    # add the complete file name
    ds = ds.map(lambda x: {'file_name':BASE_PATH+"/"+dataset_name+"/"+x["file_name"]})
    return ds['train']

In [16]:
# wav_to_base64string(fleurs_dataset['path'][0])

In [18]:
display.Audio(fleurs_dataset['path'][10])

In [19]:
requests.request("POST", url, headers=headers, data=get_payload(wav_to_base64string(fleurs_dataset['path'][10])))
ast.literal_eval(response.text)

{'status': 'SUCCESS', 'output': [{'source': ''}]}

{'status': 'SUCCESS', 'output': [{'source': ''}]}

In [11]:
from tqdm import tqdm
indicwav2vec_texts = []
for file in tqdm(fleurs_dataset['path'][:2]):
    encoded_string = wav_to_base64string(file)
    response = requests.request("POST", url, headers=headers, data=get_payload(encoded_string))
    indicwav2vec_texts.append(ast.literal_eval(response.text)['output'][0]['source'])
indicwav2vec_texts    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.67s/it]


['', '']